In [1]:
#!/usr/bin/env python
# Copyright 2023 Z Zhang

# BioSeq2Seq, Version 1.0;
# you may not use this file except in compliance with the License.
# Use of this code requires following originality guidelines
# and declaring the source of the code.
# email:zhichunli@mail.dlut.edu.cn
# =========================================================================

import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"
print("CUDA_VISIBLE_DEVICES =", os.environ.get("CUDA_VISIBLE_DEVICES"))
from re import I
import numpy as np
from model.model_MambaHM import MambaHM
from utils.dataloader import RD_dataloader
from utils.functions import bw_2_chromosome_size
from torch.utils.data import DataLoader
from numpy.core.multiarray import scalar
from tqdm import tqdm
from typing import Optional
from torchmetrics import Metric
from einops import rearrange
from operator import itemgetter
import subprocess

import torch
DEVICE=torch.device("cuda:0")

print("CUDA_VISIBLE_DEVICES =", os.environ["CUDA_VISIBLE_DEVICES"])
print("Torch sees", torch.cuda.device_count(), "GPUs")
print("Device 0 name:", torch.cuda.get_device_name(0))

CUDA_VISIBLE_DEVICES = 0


/local1/zzx/conda/envs/LongSeq/lib/python3.9/site-packages/mamba_ssm/ops/selective_scan_interface.py:164: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, xz, conv1d_weight, conv1d_bias, x_proj_weight, delta_proj_weight,
/local1/zzx/conda/envs/LongSeq/lib/python3.9/site-packages/mamba_ssm/ops/selective_scan_interface.py:240: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, dout):
/local1/zzx/conda/envs/LongSeq/lib/python3.9/site-packages/mamba_ssm/ops/triton/layer_norm.py:986: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(
/local1/zzx/conda/envs/LongSeq/lib/python3.9/site-packages/mamba_ssm/ops/triton/layer_norm.py:1045: FutureWarning: `torch.cuda.amp.custom_bwd(arg

CUDA_VISIBLE_DEVICES = 0
Torch sees 1 GPUs
Device 0 name: NVIDIA GeForce RTX 3090


Evaluation Metric

In [3]:
'''
evaluation
'''
def write_predicted_result(
                        results,
                        out_path,
                        chr_length,
                        target_list,
                        reference_genome_idx,
                        seq_length=114688,
                        window_size=128,
                        ):
    """ 
    Write result to bigwig file

    Args:
        results: predicted result, {chr:[{start:xx, end:xx, result:xx}]}
        out_path: output path
        chr_length: chromosome length
        target_list: target sequencing data list
        reference_genome_idx: reference genome idx

    Return:
        None
    """
    seq_length = seq_length
    target_length = seq_length // window_size
    
    for j in range(len(target_list)):
            if os.path.isfile(os.path.join(out_path, target_list[j] + '.bedgraph')):
                os.remove(os.path.join(out_path, target_list[j] + '.bedgraph'))

    for chr in results:
        chr_result = results[chr]
        chr_result = sorted(chr_result, key=itemgetter('start'))
        for j in range(len(target_list)):
            with open(os.path.join(out_path, target_list[j] + '.bedgraph'), 'a') as w_obj:
                # assign 0 to the area not covered by the sample
                if chr_result[0]['start'] > 0:
                    w_obj.write(chr + '\t' + str(0) + '\t' + str(chr_result[0]['start']) + '\t' + str(0) + '\n')

                # write predict result
                last_end = 0
                for item in chr_result:
                    if item['start'] >= last_end: 
                        for i in range(target_length):
                            start = item['start'] + i * window_size
                            end = start + window_size
                            w_obj.write(chr + '\t' + str(start) + '\t' + str(end) + '\t' + str(item['predicted'][i][j]) + '\n')
                    else:
                        print(item)
                        gap_h = last_end - item['start']
                        h_start = gap_h // window_size
                        w_obj.write(chr + '\t' + str(last_end) + '\t' + str(item['start'] + window_size * (h_start+1)) + '\t' + str(item['predicted'][h_start][j]) + '\n')
                        for i in range(h_start+1, target_length):
                            start = item['start'] + i * window_size
                            end = start + window_size 
                            w_obj.write(chr + '\t' + str(start) + '\t' + str(end) + '\t' + str(item['predicted'][i][j]) + '\n')
                last_end = item['end']

                # assign 0 to the area not covered by the sample
                if chr_result[-1]['end'] < chr_length[chr]:
                    w_obj.write(chr + '\t' + str(chr_result[-1]['end']) + '\t' + str(chr_length[chr]) + '\t' + str(0) + '\n')

    # bedgraph to bigwig
    for j in range(len(target_list)):
        bed_path = os.path.join(out_path, target_list[j] + '.bedgraph')
        bedgraph_path_sorted = os.path.join(out_path, target_list[j] + '_sorted.bedgraph')
        cmd_bedSort = 'sort-bed ' + bed_path + ' > ' + bedgraph_path_sorted
        p = subprocess.Popen(cmd_bedSort, shell=True)
        p.wait()

        bw_path = os.path.join(out_path, target_list[j] + '.bw')

        cmd = ['bedGraphToBigWig', bedgraph_path_sorted, reference_genome_idx, bw_path]
        subprocess.call(cmd)

        cmd_rm = ['rm', '-f', bed_path]
        subprocess.call(cmd_rm)

        cmd_rm = ['rm', '-f', bedgraph_path_sorted]
        subprocess.call(cmd_rm)

    return True

class MeanPearsonCorrCoefPerChannel(Metric):
    is_differentiable: Optional[bool] = False
    higher_is_better: Optional[bool] = True
    def __init__(self, n_channels:int, dist_sync_on_step=False):
        """Calculates the mean pearson correlation across channels aggregated over regions"""
        super().__init__(dist_sync_on_step=dist_sync_on_step)
        self.reduce_dims=(0, 1)
        self.add_state("product", default=torch.zeros(n_channels, dtype=torch.float32), dist_reduce_fx="sum", )
        self.add_state("true", default=torch.zeros(n_channels, dtype=torch.float32), dist_reduce_fx="sum", )
        self.add_state("true_squared", default=torch.zeros(n_channels, dtype=torch.float32), dist_reduce_fx="sum", )
        self.add_state("pred", default=torch.zeros(n_channels, dtype=torch.float32), dist_reduce_fx="sum", )
        self.add_state("pred_squared", default=torch.zeros(n_channels, dtype=torch.float32), dist_reduce_fx="sum", )
        self.add_state("count", default=torch.zeros(n_channels, dtype=torch.float32), dist_reduce_fx="sum")

    def update(self, preds: torch.Tensor, target: torch.Tensor):
        assert preds.shape == target.shape

        self.product += torch.sum(preds * target, dim=self.reduce_dims)
        self.true += torch.sum(target, dim=self.reduce_dims)
        self.true_squared += torch.sum(torch.square(target), dim=self.reduce_dims)
        self.pred += torch.sum(preds, dim=self.reduce_dims)
        self.pred_squared += torch.sum(torch.square(preds), dim=self.reduce_dims)
        self.count += torch.sum(torch.ones_like(target), dim=self.reduce_dims)

    def compute(self):
        true_mean = self.true / self.count
        pred_mean = self.pred / self.count

        covariance = (self.product
                    - true_mean * self.pred
                    - pred_mean * self.true
                    + self.count * true_mean * pred_mean)

        true_var = self.true_squared - self.count * torch.square(true_mean)
        pred_var = self.pred_squared - self.count * torch.square(pred_mean)
        tp_var = torch.sqrt(true_var) * torch.sqrt(pred_var)
        correlation = covariance / tp_var
        return correlation
    

def convert_resolution(target, window_width, aim_resolution):
    k = aim_resolution // window_width
    target = rearrange(target, 'b (r n) d -> b r n d', n = k)
    target = torch.mean(target, dim=2)
    return target

1. Data Preparation

In [ ]:
reference_genome_file = 'hg19.fa'
sequence_data_file = [
                        [
                        ['k562.bw'],
                         ]
                        ]

target_seq_file = [
    # K562
    ['H3k122ac.bigWig',
    'H3k4me1.bigWig', 
    'H3k4me2.bigWig', 
    'H3k4me3.bigWig', 
    'H3k27ac.bigWig', 
    'H3k27me3.bigWig', 
    'H3k36me3.bigWig', 
    'H3k9ac.bigWig', 
    'H3k9me3.bigWig', 
    'H4k20me1.bigWig', ]
    ]

validation_samples = np.loadtxt('MambaHM/samples/test.bed', dtype=str, delimiter='\t')

model_dir = 'MambaHM/model'

2. Model Building

In [6]:
torch.serialization.add_safe_globals([scalar])
window_width = 16
checkpoint_path = os.path.join(model_dir, 'model.pth')
checkpoint = torch.load(checkpoint_path, map_location=DEVICE    )

model = MambaHM(channels=384,
            num_heads=8,
            num_transformer_layers=11,
            pooling_type='max',
            output_channels=len(target_seq_file[0]),
            target_length=7168,
            device=DEVICE
            ).to(DEVICE)


model.load_state_dict(checkpoint['model_state_dict'])

/tmp/ipykernel_231016/2975001650.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=DEVICE    )


<All keys matched successfully>

3. Evaluation

In [5]:
eva_data_loader = RD_dataloader(validation_samples,
                        reference_genome_file,
                        sequence_data_file,
                        target_seq_file,
                        window_width=window_width,
                        extend=40960,  
                        nan=0,
                        valid=True,
                        rc=False,
                        )
eva_dataset = DataLoader(dataset=eva_data_loader, batch_size=4, shuffle=True)

metric = MeanPearsonCorrCoefPerChannel(n_channels=10)
model.eval()
# model.set_eval()
with torch.no_grad():
    for i, eva_data_item in tqdm(enumerate(eva_dataset), ascii=True):
        # if i > 60:
        #     break
        dna = eva_data_item[0].to(DEVICE)
        seq = eva_data_item[1].to(DEVICE)
        target = eva_data_item[2].to(DEVICE)
        predicted = model(dna, seq).to(DEVICE)
        target_resolution = convert_resolution(target, window_width, 1024).detach().cpu()
        predicted_resolution = convert_resolution(predicted, window_width, 1024).detach().cpu()

        target_t = target_resolution
        predicted_t = predicted_resolution
        metric.update(target_t, predicted_t)

for pearson in metric.compute().numpy():
    print(round(pearson, 4))

108it [03:01,  1.68s/it]

0.7497
0.8404
0.9087
0.9123
0.8373
0.7186
0.8176
0.8994
0.7963
0.8766


4. predict

In [ ]:
samples = np.loadtxt('MambaHM/samples/pred_chr22.bed', dtype=str, delimiter='\t')
eva_data_loader = RD_dataloader(samples,
                        reference_genome_file,
                        sequence_data_file,
                        target_seq_file,
                        window_width=window_width,
                        extend=40960,  
                        nan=0,
                        valid=True,
                        rc=False,
                        )
eva_dataset = DataLoader(dataset=eva_data_loader, batch_size=1, shuffle=False)

target_list = [
            'H3k122ac',
            'H3k4me1',
            'H3k4me2',
            'H3k4me3',
            'H3k27ac',
            'H3k27me3',
            'H3k36me3',
            'H3k9ac',
            'H3k9me3',
            'H4k20me1',
            ]
out_path = 'MambaHM/test'
results = {}
chrom_size = bw_2_chromosome_size(sequence_data_file[0][0][0])

chr_length = {}
for chr in np.unique(samples[:, 0]):
    results[chr] = []
    chr_length[chr] = chrom_size[chr][0][1]

if not os.path.isdir(out_path):
    os.mkdir(out_path)

# chromosome length file
reference_genome_idx = os.path.join(out_path, 'idx.fai')
with open(reference_genome_idx, 'w') as w_obj:
    for chr in chrom_size:
        w_obj.write(chr + '\t' + str(chrom_size[chr][0][1]) + '\n')

metric = MeanPearsonCorrCoefPerChannel(n_channels=10)
model.eval()
with torch.no_grad():
    for i, eva_data_item in tqdm(enumerate(eva_dataset), ascii=True):
        result = {}
        if i > len(samples):
            break
        dna = eva_data_item[0].to(DEVICE)
        seq = eva_data_item[1].to(DEVICE)
        target = eva_data_item[-1].to(DEVICE)
        predicted = model(dna, seq).to(DEVICE)

        predict_cpu = predicted.detach().cpu()
        result['chr'] = samples[i][0]
        result['start'] = int(samples[i][1])
        result['end'] = int(samples[i][2])
        result['predicted'] = predict_cpu[0].numpy()
        results[result['chr']].append(result)

write_down = write_predicted_result(results, out_path, chr_length, target_list, reference_genome_idx, seq_length=114688, window_size=16)

print('Prediction completed. Results saved to %s' % out_path)





0it [00:00, ?it/s]

428it [01:12,  5.94it/s]


Prediction completed. Results saved to /local1/zzx/code/LongSeq/MambaHM/test
